<a href="https://colab.research.google.com/github/shazzad-hasan/training-reproducible-deep-learning-models/blob/main/AlexNet/alexnet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/shazzad-hasan/training-reproducable-deep-learning-models.git

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
%cd /content/training-reproducable-deep-learning-models/AlexNet

In [ ]:
!ls

In [ ]:
# import required libraries
import torch
import torchvision 

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import local helper functions
from helper_dataset import dataloader_cifar10
from helper_train import train
from helper_evaluate import compute_loss_accuracy, set_all_seeds
from helper_plot import show_examples, plot_results, show_sample_test_result

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

In [ ]:
# settings
random_seed = 123
valid_size = 0.2
batch_size = 32
num_epochs = 5

set_all_seeds(random_seed)

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

train_transforms = torchvision.transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      ])

test_transforms = torchvision.transforms.Compose([
    transforms.Resize((227, 227)),        
    transforms.ToTensor(),                
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_loader, valid_loader, test_loader, classes = dataloader_cifar10(
    batch_size = batch_size, 
    train_transform = train_transforms,
    test_transform = test_transforms,
    valid_size = valid_size
)

In [ ]:
print(classes)
num_classes = len(classes)

In [ ]:
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

In [ ]:
import torch.nn as nn

class AlexNet(nn.Module):
  def __init__(self, num_classes): 
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), 256 * 6 * 6)
    logits = self.classifier(x)
    return logits

model = AlexNet(num_classes=10)
model.to(device)

print(model)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() 
params = model.parameters()
optimizer = optim.SGD(params, lr=0.01, momentum=0.9)

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, mode="min")

In [ ]:
train_loss_list, valid_loss_list, train_acc_list, valid_acc_list = train(model, num_epochs, 
                                                                         train_loader, valid_loader,
                                                                         test_loader, optimizer, 
                                                                         criterion, device)

In [ ]:
plot_results(train_loss_list, valid_loss_list, train_acc_list, valid_acc_list, num_epochs)

In [ ]:
model.load_state_dict(torch.load('model.pt'))

In [ ]:
test_loss, test_acc = compute_loss_accuracy(model, test_loader, criterion, num_classes, classes, device)

In [ ]:
show_sample_test_result(test_loader, model, classes, train_on_gpu, device)